In [4]:
import requests
import pandas as pd 
from tqdm.notebook import tqdm

In [2]:
api_key = None

In [3]:
markets = [
    'de_DE',
    'en_US',
]

headers = {
    'accept': 'application/json',
    'x-api-key': api_key,
}

In [5]:
data = {
    'market_id': [],
    'model_id': [],
    'type_class': [],
    'name': [],
    'short_name': [],
    'brand': [],
    'baumuster': [],
    'class': [],
    'body': [],
    'model_year': [],
    'product_group': [],
    'all_terrain': [],
    'currency': [],
    'net_price': [],
    'price': [],
    'steering_position': [],
}

save_all_data = {}

for market_id in markets:
    url = f'https://api.mercedes-benz.com/configurator/v2/markets/{market_id}/bodies'

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        bodies = response.json()
    else:
        raise Exception(f'Body request {url} failed')
    
    save_all_data[market_id] = {'bodies': bodies}
    bodies = [body['bodyId'] for body in bodies]

    for body_id in bodies:
        url = f'https://api.mercedes-benz.com/configurator/v2/markets/{market_id}/models?bodyId={body_id}'

        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            models = response.json()
        else:
            raise Exception(f'Models request {url} failed')

        print(market_id, body_id)
        for model in models:
            data['market_id'].append(market_id)
            data['model_id'].append(model['modelId'])
            data['type_class'].append(model['typeClass'])
            data['name'].append(model['name'])
            data['short_name'].append(model['shortName'])
            data['brand'].append(model['brand']['name'])
            data['baumuster'].append(model['baumuster'])
            data['class'].append(model['vehicleClass']['className'])
            data['body'].append(model['vehicleBody']['bodyName'])
            data['model_year'].append(model['modelYear'])
            data['product_group'].append(model['productGroup'])
            data['all_terrain'].append(model['allTerrain'])
            data['currency'].append(model['priceInformation']['currency'])
            data['net_price'].append(model['priceInformation']['netPrice'])
            data['price'].append(model['priceInformation']['price'])
            data['steering_position'].append(model['steeringPosition'])

de_DE KOMPAKT-LIMOUSINE
de_DE LIMOUSINE
de_DE COUPE
de_DE T-MODELL
de_DE OFFROADER
de_DE GELAENDEWAGEN
de_DE ROADSTER
de_DE LIMOUSINE_LANG
de_DE 4-TUERER COUPE
de_DE SPORTS TOURER
de_DE SHOOTING BRAKE
de_DE CABRIOLET
de_DE MAYBACH
de_DE CHASSIS
de_DE CHASSIS-CC
de_DE PANEL-VAN
de_DE TOURER
de_DE PANEL-VAN
de_DE PEOPLE-CARRIER
de_DE TOURER
de_DE TOURER
de_DE MIXTO
de_DE PEOPLE-CARRIER
de_DE MARCO-POLO
de_DE PANEL-VAN
en_US CABRIOLET
en_US COUPE
en_US LIMOUSINE
en_US OFFROADER
en_US ROADSTER
en_US LIMOUSINE_LANG
en_US 4-TUERER COUPE
en_US T-MODELL
en_US GELAENDEWAGEN
en_US MAYBACH
en_US CHASSIS
en_US PANEL-VAN
en_US TOURER


In [6]:
df = pd.DataFrame(data)

In [7]:
df_ev = pd.concat([
    df.query('name.str.contains("EQ")'),
    df.query('name.str.contains(" E ")'),
    df.query('name.str.contains("HYBRID")'),
]).drop_duplicates()


In [9]:
df_ev.to_csv('mercedes_ev.csv', index=False)

# Experiments

In [ ]:
1 / 0

In [95]:
language = 'en'

url = f'https://api.mercedes-benz.com/configurator/v2/markets?language={language}'

headers = {
    'accept': 'application/json',
    'x-api-key': api_key,
}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    markets = response.json()
else:
    print("Request failed with status code:", response.status_code)

In [7]:
market_id = markets[0]['marketId']

In [8]:
url = f'https://api.mercedes-benz.com/configurator/v2/markets/{market_id}/bodies'

response = requests.get(url, headers=headers)
if response.status_code == 200:
    bodies = response.json()
else:
    print("Request failed with status code:", response.status_code)

In [12]:
body_id = bodies[2]['bodyId']

In [13]:
url = f'https://api.mercedes-benz.com/configurator/v2/markets/{market_id}/models?bodyId={body_id}'

response = requests.get(url, headers=headers)
if response.status_code == 200:
    models = response.json()
else:
    print("Request failed with status code:", response.status_code)

In [20]:
model_id = models[0]['modelId']

In [24]:
url = models[0]['_links']['configurations']

response = requests.get(url, headers=headers)
if response.status_code == 200:
    configurations = response.json()
else:
    print("Request failed with status code:", response.status_code)

In [27]:
url = configurations['_links']['selectables']
response = requests.get(url, headers=headers)
if response.status_code == 200:
    selectables = response.json()
else:
    print("Request failed with status code:", response.status_code)

In [41]:
conf_id = configurations['configurationId']

In [61]:
url = f'https://api.mercedes-benz.com/configurator/v2/markets/{market_id}/models/{model_id}/configurations/{conf_id}/images/vehicle?perspectives=EXT020%2CINT1&roofOpen=false&night=false&background=true'

response = requests.get(url, headers={'x-api-key': api_key})
if response.status_code == 200:
    imgs = response.json()
else:
    print("Request failed with status code:", response.status_code)


In [62]:
imgs['vehicle']['EXT020']['url']

'https://europe.starconnect-ce.i.mercedes-benz.com/iris/iris.webp?COSY-EU-100-1713d0VXqNEFqtyO67PobzIr3eWsrrCsdRRzwQZQ9vZbMw3SGtlaWtsd2HVcUfpOkXGEubSJ0l3IrOB2NS1bApj7oI5ux5xQC31CpkzNwPnm7jZeDhKV5pV%25vq4tTyLRgLFYaxPrWrH1eOdn8w0Q3oiZBklM4FAyrTg95Ye6PDakDSeWHXutsd8ZDcUfiMcXGE4JmJ0lgOrOB2Pz%25bApe7pI5usKDQC3vM7kzNLt6m7jaSthymI9WFAocUf8FfXGEH97J0lU9YOB2GyrbApF0MI5uC4zQC3zs3kzN7%259m7jKh3hKVvHW%25vqJtdyL0fsjko9wdYfHJyCiNRcYxqN88Zlo21Ix2rbpldCdcqcgVGB4NPVHF88iu&&BKGND=12&IMGT=W27&POV=BE020,PZM'

In [55]:
vehicle_id = configurations['vehicleId']

In [56]:
url = f'https://api.mercedes-benz.com/configurator/v2/stocksearch/vehicles/{vehicle_id}'
response = requests.get(url, headers=headers)
if response.status_code == 200:
    stocks = response.json()
else:
    print("Request failed with status code:", response.status_code)


Request failed with status code: 500


In [60]:
api_key

'f59b1db4-e310-4f7c-8ceb-8584ea3c8465'

In [59]:
!curl -X GET "$url" -H "accept: application/json" -H "x-api-key:$api_key"


                        {
                        "errorMessage": "Server Error",
                        "statusCode": "500"
                        }
                    

In [65]:
url = f'https://api.mercedes-benz.com/configurator/v2/vehicle-models/markets/{market_id}'

response = requests.get(url, headers={'x-api-key': api_key})
if response.status_code == 200:
    detailed = response.json()
else:
    print("Request failed with status code:", response.status_code)


Request failed with status code: 204


'f59b1db4-e310-4f7c-8ceb-8584ea3c8465'

In [73]:
models[0]

{'modelId': '2363621',
 'typeClass': 'C236',
 'name': 'Mercedes-AMG CLE 53 4MATIC+ Coupé',
 'shortName': 'Mercedes-AMG CLE 53 4MATIC+ Coupé',
 'brand': {'name': 'Mercedes-AMG'},
 'baumuster': '2363621',
 'baumuster6': '236362',
 'baureihe': '236',
 'nationalSalesType': '',
 'vehicleClass': {'classId': 'CLE-KLASSE',
  'className': 'CLE',
  '_links': {'self': 'https://api.mercedes-benz.com/configurator/v2/markets/de_DE/classes/CLE-KLASSE',
   'models': 'https://api.mercedes-benz.com/configurator/v2/markets/de_DE/models?classId=CLE-KLASSE'}},
 'vehicleBody': {'bodyId': 'COUPE',
  'bodyName': 'Coupe',
  '_links': {'self': 'https://api.mercedes-benz.com/configurator/v2/markets/de_DE/bodies/COUPE',
   'models': 'https://api.mercedes-benz.com/configurator/v2/markets/de_DE/models?bodyId=COUPE'}},
 'modelYear': '804',
 'productGroup': 'PKW',
 'productDivision': 0,
 'lifeCycle': 'New model',
 'facelift': False,
 'allTerrain': False,
 'customerGroups': ['PRIVATE'],
 'priceInformation': {'currency

In [78]:
baumuster = models[0]['baumuster']
model_year = models[0]['modelYear']

In [79]:
url = f'https://api.mercedes-benz.com/configurator/v2/vehicle-models/markets/{market_id}/baumuster/{baumuster}?modelYear={model_year}'
response = requests.get(url, headers=headers)
if response.status_code == 200:
    detailed = response.json()
else:
    print("Request failed with status code:", response.status_code)


Request failed with status code: 404
